<svg width="300" height="100" xmlns="http://www.w3.org/2000/svg">
<line x1="10" y1="10" x2="290" y2="10" style="stroke:#000">
</line>
<ellipse cx="150" cy="50" rx="20" ry="20" style="stroke:#000;fill:none;"></ellipse>
</svg>

In [ ]:
from IPython.display import SVG
import svgwrite
from xml.dom import minidom
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import os
import random
import pandas as pd
from random import randint
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
import warnings

In [ ]:
def parse_path(path,output,start=[None,None],last_key=None,offset=[0,0],first_run=True):
    while len(path) > 0:
        key = path.pop(0)
        if key == 'm':
            if first_run:
                start = list(map(float,path.pop(0).split(',')))
                x,y = 0 + offset[0], 0 + offset[1]
                first_run=False
            else:
                x,y = list(map(float,path.pop(0).split(',')))
            output.append(['m',0,x,y,0,0,0,0])
            last_key = "m"
        elif key == 'M':
            if first_run:
                start = list(map(float,path.pop(0).split(',')))
                x,y = 0 + offset[0],0 + offset[1]
                first_run=False
            else:
                x,y = list(map(float,path.pop(0).split(',')))
                x = x- start[0] + offset[0]
                y = y- start[1] + offset[1]
            output.append(['m',1,x,y,0,0,0,0])
            last_key = "M"
        elif key == 'c':
            x1,y1 = list(map(float,path.pop(0).split(',')))
            x2,y2 = list(map(float,path.pop(0).split(',')))
            x,y = list(map(float,path.pop(0).split(',')))
            output.append(['c',0,x1,y1,x2,y2,x,y])
            last_key = "c"
        elif key == 'C':
            x1,y1 = list(map(float,path.pop(0).split(',')))
            x1 = x1 - start[0] + offset[0]
            y1 = y1 - start[1] + offset[1]
            x2,y2 = list(map(float,path.pop(0).split(',')))
            x2 = x2 - start[0] + offset[0]
            y2 = y2 - start[1] + offset[1]
            x,y = list(map(float,path.pop(0).split(',')))
            x = x - start[0] + offset[0]
            y = y - start[1] + offset[1]
            output.append(['c',1,x1,y1,x2,y2,x,y])
            last_key = "C"
        elif key == 'z' or key == 'Z':
            output.append(['z',0,0,0,0,0,0,0])
            last_key = "z"
        elif key == 'l':
            x,y = list(map(float,path.pop(0).split(',')))
            output.append(['l',0,x,y,0,0,0,0])
            last_key = "l"
        elif key == 'L':
            x,y = list(map(float,path.pop(0).split(',')))
            x = x - start[0] + offset[0]
            y = y - start[1] + offset[1]
            output.append(['l',1,x,y,0,0,0,0])
            last_key = "L"
        elif key == 'h':
            x = float(path.pop(0))
            output.append(['l',0,x,0,0,0,0,0]) #l is ententionally put here to minimize the # of commands 
            last_key = "h"
        elif key == 'H':
            x = float(path.pop(0))
            x = x - start[0] + offset[0]
            output.append(['l',1,x,0,0,0,0,0]) #l is ententionally put here to minimize the # of commands
            last_key = "H"
        elif key == 'v':
            y = float(path.pop(0))
            output.append(['l',0,0,y,0,0,0,0]) #l is ententionally put here to minimize the # of commands
            last_key = "v"
        elif key == 'V':
            y = float(path.pop(0))
            y = y - start[1] + offset[1]
            output.append(['l',1,0,y,0,0,0,0]) #l is ententionally put here to minimize the # of commands 
            last_key = "V"
        else:
            if last_key == 'c':
                x1,y1 = list(map(float,key.split(',')))
                x2,y2 = list(map(float,path.pop(0).split(',')))
                x,y = list(map(float,path.pop(0).split(',')))
                output.append(['c',0,x1,y1,x2,y2,x,y])
                last_key = "c"
            elif last_key == 'C':
                x1,y1 = list(map(float,key.split(',')))
                x1 = x1 - start[0] + offset[0]
                y1 = y1 - start[1] + offset[1]
                x2,y2 = list(map(float,path.pop(0).split(',')))
                x2 = x2 - start[0] + offset[0]
                y2 = y2 - start[1] + offset[1]
                x,y = list(map(float,path.pop(0).split(',')))
                x = x - start[0] + offset[0]
                y = y - start[1] + offset[1]
                output.append(['c',1,x1,y1,x2,y2,x,y])
                last_key = "C"
            elif last_key == 'm':
                x,y = list(map(float,key.split(',')))
                output.append(['m',0,x,y,0,0,0,0])
                last_key = "m"
            elif last_key == 'M':
                x,y = list(map(float,key.split(',')))
                x = x- start[0] + offset[0]
                y = y- start[1] + offset[1]
                output.append(['m',1,x,y,0,0,0,0])
                last_key = "M"
            elif last_key == 'l':
                x,y = list(map(float,key.split(',')))
                output.append(['l',0,x,y,0,0,0,0])
                last_key = "l"
            elif last_key == 'L':
                x,y = list(map(float,key.split(',')))
                x = x - start[0] + offset[0]
                y = y - start[1] + offset[1]
                output.append(['l',1,x,y,0,0,0,0])
                last_key = "L"
            elif last_key == 'h':
                x = float(key)
                output.append(['l',0,x,0,0,0,0,0]) #l is ententionally put here to minimize the # of commands 
                last_key = "h"
            elif last_key == 'H':
                x = float(key)
                x = x - start[0] + offset[0]
                output.append(['l',1,x,0,0,0,0,0]) #l is ententionally put here to minimize the # of commands
                last_key = "H"
            elif last_key == 'v':
                y = float(key)
                output.append(['l',0,0,y,0,0,0,0]) #l is ententionally put here to minimize the # of commands
                last_key = "v"
            elif last_key == 'V':
                y = float(key)
                y = y - start[1] + offset[1]
                output.append(['l',1,0,y,0,0,0,0]) #l is ententionally put here to minimize the # of commands
                last_key = "V"
            else:
                print("Undefined propagated key",key,last_key)
    return np.array(output,dtype=object)

In [ ]:
# Prior to running the function, one-hot encoded values should be brough back to letters for simplicity! 
def generate_path(commands,path=''):
    for command in commands:
        if command[0] == 'm':
            if command[1] == 0:
                path += 'm ' + str(command[2]) + ',' + str(command[3]) + ' '
            elif command[1] == 1:
                path += 'M ' + str(command[2]) + ',' + str(command[3]) + ' '
            else:
                print('Unknown state')
        elif command[0] == 'c':
            if command[1] == 0:
                path += 'c ' + str(command[2]) + ',' + str(command[3]) + ' ' + str(command[4]) + ',' + str(command[5])  + ' ' + str(command[6]) + ',' + str(command[7]) + ' '
            elif command[1] == 1:
                path += 'C ' + str(command[2]) + ',' + str(command[3]) + ' ' + str(command[4]) + ',' + str(command[5])  + ' ' + str(command[6]) + ',' + str(command[7]) + ' '
            else:
                print('Unknown state')
        elif command[0] == 'z':
            path += command[0] + ' '
        elif command[0] == 'l':
            if command[1] == 0:
                path += 'l ' + str(command[2]) + ',' + str(command[3]) + ' '
            elif command[1] == 1:
                path += 'L ' + str(command[2]) + ',' + str(command[3]) + ' '
            else:
                print('Unknown state')        
        else:
            print('Unknown command, returning the partial path')
    return path[:-1]

In [ ]:
def normalize(matrix):
    matrix[:,2:] = ((np.abs(matrix[:,2:]) - np.min(np.abs(matrix[:,2:])))/(np.max(np.abs(matrix[:,2:])) - np.min(np.abs(matrix[:,2:]))))*np.sign(matrix[:,2:])
    return matrix

In [ ]:
def rotate(norm_input,angle):
    r_matrix = np.array([[np.cos(angle),-np.sin(angle)],[np.sin(angle),np.cos(angle)]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)      

In [ ]:
def flip_x(norm_input):
    r_matrix = np.array([[1,0],[0,-1]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)      

In [ ]:
def flip_y(norm_input):
    r_matrix = np.array([[-1,0],[0,1]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)      

In [ ]:
def flip_o(norm_input):
    r_matrix = np.array([[-1,0],[0,-1]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)         

In [ ]:
def flip_xy(norm_input):
    r_matrix = np.array([[0,1],[1,0]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)

In [ ]:
def scale_x(norm_input,scale_x):
    r_matrix = np.array([[scale_x,0],[0,1]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)

In [ ]:
def scale_y(norm_input,scale_y):
    r_matrix = np.array([[1,0],[0,scale_y]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)

In [ ]:
def scale_xy(norm_input,scale_x,scale_y):
    r_matrix = np.array([[scale_x,0],[0,scale_y]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)

In [ ]:
def skew_x(norm_input,scale_x):
    r_matrix = np.array([[1,scale_x],[0,1]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)

In [ ]:
def skew_y(norm_input,scale_y):
    r_matrix = np.array([[1,0],[scale_y,1]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)

In [ ]:
def skew_xy(norm_input,scale_x,scale_y):
    r_matrix = np.array([[1,scale_x],[scale_y,1]])
    return np.apply_along_axis(lambda x: np.hstack([x[0:2],np.dot(x[2:4],r_matrix),np.dot(x[4:6],r_matrix),np.dot(x[6:8],r_matrix)]),1,norm_input)    

In [ ]:
def apply_all_flips(norm_output,samples):
    #samples.append(flip_x(norm_output))
    samples.append(flip_y(norm_output))
    samples.append(flip_o(norm_output))
    #samples.append(flip_xy(norm_output))
    return samples

In [ ]:
def apply_all_skews(norm_output,num_transform,samples,multiplier):
    x_scales = np.random.uniform(-0.7,0.7, num_transform)
    y_scales = np.random.uniform(-0.7,0.7, num_transform)
    x_scales_2 = np.random.uniform(-0.5,0.5, num_transform) #we could leverage the same x and y, though
    y_scales_2 = np.random.uniform(-0.5,0.5, num_transform)
    for i in range(num_transform):
        skewed = skew_x(norm_output,x_scales[i])
        max_move = max(abs(np.min(skewed[:,1:])),np.max(skewed[:,1:]))
        #get necessary scaling
        skew_scale = multiplier/max_move
        if skew_scale < 1:
            skewed = scale_xy(skewed,skew_scale,skew_scale)
        samples.append(skewed)
        skewed = skew_y(norm_output,y_scales[i])
        max_move = max(abs(np.min(skewed[:,1:])),np.max(skewed[:,1:]))
        #get necessary scaling
        skew_scale = multiplier/max_move
        if skew_scale < 1:
            skewed = scale_xy(skewed,skew_scale,skew_scale)
        samples.append(skewed)
        skewed = skew_xy(norm_output,x_scales_2[i],y_scales_2[i])
        max_move = max(abs(np.min(skewed[:,1:])),np.max(skewed[:,1:]))
        #get necessary scaling
        skew_scale = multiplier/max_move
        if skew_scale < 1:
            skewed = scale_xy(skewed,skew_scale,skew_scale)
        samples.append(skewed)     
    return samples

In [ ]:
def apply_all_scales(norm_output,num_transform,samples,multiplier):
    x_scales = np.random.uniform(0.5,1, num_transform)
    y_scales = np.random.uniform(0.5,1, num_transform)
    x_scales_2 = np.random.uniform(0.5,1, num_transform) #we could leverage the same x and y, though
    y_scales_2 = np.random.uniform(0.5,1, num_transform)
    for i in range(num_transform):
        scaled = scale_x(norm_output,x_scales[i])
        samples.append(scaled)
        samples = apply_all_skews(scaled,num_transform,samples,multiplier)
        scaled = scale_y(norm_output,y_scales[i])
        samples.append(scaled)
        samples = apply_all_skews(scaled,num_transform,samples,multiplier)
        scaled = scale_xy(norm_output,x_scales_2[i],y_scales_2[i])
        samples.append(scaled)
        samples = apply_all_skews(scaled,num_transform,samples,multiplier)        
    return samples    

In [ ]:
#rotate,flip_x,flip_y,flip_o,flip_xy,scale_x,scale_y,scale_xy,skew_x,skew_y,skew_xy
def random_transform(norm_output,num_transform,samples,multiplier):
    #We wanna combine transformations as well, flip in all possible ways, rotate, scale, skew, then flip again
    #Flip the original image,
    samples.append(norm_output)
    return samples
    samples = apply_all_flips(norm_output,samples)
    sample_len = len(samples)
    for i in range(sample_len):
        #generate random angles (-180,180)
        angles = random.sample(range(0,1), 1) #updated 
        for angle in angles:
            rotated = rotate(samples[i],angle)
            #pass the multiplier here, get the max after rotation 
            max_move = max(abs(np.min(rotated[:,1:])),np.max(rotated[:,1:]))
            #get necessary scaling
            rotation_scale = multiplier/max_move
            if rotation_scale < 1:
                rotated = scale_xy(rotated,rotation_scale,rotation_scale)
            samples.append(rotated)
            samples = apply_all_scales(rotated,num_transform,samples,multiplier)
    return samples

In [ ]:
def split_and_fill(sample,length,split_multiplier):
    if len(sample) < length:
        return np.expand_dims(np.vstack((sample,np.zeros((length-len(sample),sample.shape[1])))),axis=0)
    elif len(sample) > length:
        n_splits = int((len(sample)/length-1)*split_multiplier)+1
        split_points = random.sample(range(len(sample)-length),n_splits)
        subsamples = []
        for point in split_points:
            subsamples.append(sample[point:point+length])
        return np.array(subsamples)
    else:
        return np.expand_dims(sample,axis=0)

In [ ]:
doc = minidom.parse("Minisample/flowers.svg")  # parseString also exists geometrical_4
square_path = [path.getAttribute('d') for path
                in doc.getElementsByTagName('path')]
doc.unlink()
square_matrix = parse_path(square_path[4].split(),output=[],offset=[0,0],first_run=True)
square_matrix.shape

In [ ]:
square_matrix.shape

In [ ]:
#transformed_samples = random_transform(square_matrix,10,[],50)

In [ ]:
#path_6 = generate_path(scale_x(small_norm[:],0.5))
path_6 = generate_path(rotate(flip_x(square_matrix),0))
dwg_6 = svgwrite.Drawing('test_6.svg') #,size = ("247", "148")
#dwg_4.viewbox(0,0,1000,100)
dwg_6.add(dwg_6.path( d=path_6,
    transform="scale(0.9), matrix(1.3333333,0,0,-1.3333333,100,100)"))
#))
dwg_6.save()
SVG('test_6.svg')

In [ ]:
#all_results = np.vstack(transformed_samples).reshape(-1,10,8)

In [ ]:
def prepare_data(min_length=5,seq_length=57,multiplier=50,discrete=False,num_transform=2):
    # Spacers will go with all 0s
    # We wanna one hot encode the first column before forming the last matrix
    all_results = []
    for filename in os.listdir('Minisample'):
        doc = minidom.parse(os.path.join('Minisample',filename))
        path_strings = [path.getAttribute('d') for path
                in doc.getElementsByTagName('path')]
        doc.unlink()
        parsed = [] 
        for i,path in enumerate(path_strings):
            try:
                parsed_matrix = parse_path(path.split(),output=[],offset=[0,0],first_run=True)
            except:
                print("parse error is in",i)
            if len(parsed_matrix) < min_length:
                continue
            norm_matrix = normalize(parsed_matrix)
            norm_matrix[:,2:] = norm_matrix[:,2:]*multiplier
            #do the transformations
            transformed_samples = random_transform(norm_matrix,num_transform,[],multiplier)
            #fill and cut
            for sample in transformed_samples:
                #sample = flip_x(sample)
                if discrete:
                    sample[:,2:] = np.apply_along_axis(lambda x: list(map(int,x)),1,sample[:,2:])
                subsamples = split_and_fill(sample,seq_length,2)
                parsed.append(subsamples)
        #print(parsed[0].shape,parsed[1].shape)
        all_results.append(np.vstack(parsed))
        try:
            print(filename,":",np.min(list(map(len,parsed))),np.max(list(map(len,parsed))),np.median(list(map(len,parsed))))
        except:
            print(parsed)
        break
    return np.vstack(all_results)   

In [ ]:
all_results = prepare_data(num_transform=20)

In [ ]:
all_results.shape

In [ ]:
test_cov = all_results[:,:,2:].reshape((-1,6))
cov = np.cov(test_cov.T.astype('float'))
cov = torch.tensor(cov,dtype=torch.float)

In [ ]:
label_enc = LabelEncoder()
labels = label_enc.fit_transform(list(all_results[:,:,0].reshape(-1, 1).flatten()))
labels = np.expand_dims(labels,axis=1) #do we need this one?
labels.shape
labels = labels.reshape((-1,57))
all_results[:,:,0] = labels

In [ ]:
# testing the one hot encoding part
#norm_output = all_results[0]
enc = OneHotEncoder(handle_unknown='ignore')
encodings = enc.fit_transform(all_results[:,:,0].reshape(-1, 1)).toarray()
encoded_values = np.hstack((encodings,all_results[:,:,1:].reshape(encodings.shape[0],all_results.shape[2]-1)))
encoded_values = encoded_values.reshape((all_results.shape[0],all_results.shape[1],encoded_values.shape[1]))

In [ ]:
print(all_results.shape,encoded_values.shape) # should be (54375, 100, 8) (54375, 100, 12)

In [ ]:
print(np.min(encoded_values),np.max(encoded_values)) #We don't need scaling under 50 anymore

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
x = encoded_values[:,:-1,:]
print(x.shape) #(54375, 99, 12)
x = torch.tensor(x.astype('float'),dtype=torch.float,device=cuda0) #,device=cuda0

In [ ]:
y = all_results[:,1:,:]
print(y.shape) #(54375, 99, 8)
y = torch.tensor(y.astype('float'),dtype=torch.float,device=cuda0) #,device=cuda0

In [ ]:
x_batches = torch.split(x,50)

In [ ]:
y_batches = torch.split(y,50)

In [ ]:
class LSTMArtist(nn.Module):

    def __init__(self, input_dim, hidden_dim, class_dim, mix_k, num_layers=2,dropout=0.1): #, dropout=0.1
        super(LSTMArtist, self).__init__()
        
        #self.cov = torch.eye(6)
        #self.cov.requires_grad = True
        
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers=num_layers, dropout=dropout, batch_first=True)
        self.hidden2method = nn.Linear(hidden_dim, mix_k*class_dim)
        self.hidden2absolute = nn.Linear(hidden_dim, mix_k)
        #self.hidden2coordinates = nn.Linear(hidden_dim, 6)
        
        self.linear_mu = nn.Linear(hidden_dim,mix_k*6)
        self.linear_sigmasq = nn.Linear(hidden_dim,mix_k*36)
        self.linear_mix = nn.Linear(hidden_dim,mix_k)
        
        self.activation = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=2)
        self.relu = nn.ReLU()


    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        method = self.softmax(self.hidden2method(lstm_out))
        absolute = self.sigmoid(self.hidden2absolute(lstm_out)).squeeze()
        #coordinates = self.hidden2coordinates(lstm_out)
        coordinates_mu = self.linear_mu(lstm_out)
        coordinates_sigmasq = torch.exp(self.linear_sigmasq(lstm_out)) #torch.exp self.relu
        coordinates_mix = self.softmax(self.linear_mix(lstm_out))
        return (method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix)

In [ ]:
def gaussian_pdf(x, u, sigmasq):
    # NOTE: we could use the new `torch.distributions` package for this now
    #print((-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2)
    #print(torch.exp((-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2))
    #get the multivariate function here and use the diagonal matrix
    k = x.shape[-1]
    #cov = torch.diag_embed(sigmasq)
    cov = sigmasq.reshape((sigmasq.shape[0],sigmasq.shape[1],6,6))
    #male the matrix positive-definite
    cov = torch.matmul(cov,cov.permute(0,1,3,2))
    #print(cov.shape)
    cov = cov + torch.eye(6,device=cuda0).repeat(x.shape[0],x.shape[1],1,1)

    #print(cov)
    t1 = (2 * np.pi)**k
    t2 = torch.det(cov)
    t3 = 1.0 / torch.sqrt(t1 * t2)  # scalar 
    t4 = (x - u)
    #print("t1:",t1)
    #print("t2:",t2)
    #print("t3:",t1 * t2)
    #print("t4:",t4)
    t5 = torch.inverse(cov)
    #print("t5:",t5)
    t7 = -0.5 * torch.diagonal(torch.matmul(torch.diagonal(torch.matmul(t4.reshape((-1,k)),t5.reshape(-1,k,k)),dim1=0,dim2=1).T,t4.reshape((-1,k)).T),dim1=0,dim2=1).reshape((t4.shape[0:2]))
    pdf = t3 * torch.exp(torch.min(torch.tensor(50.0).to(cuda0),torch.max(torch.tensor(-50.0).to(cuda0),t7))) 
    return pdf

In [ ]:
def calculate_loss(method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix,t,k,y,ce_loss,bce_loss,methods_n):
    method = method.permute((0,2,1))
    #print(out_cordinates.shape)
    y_method = y[:,:,0].long()
    y_absolute = y[:,:,1]
    #y_coordinates = y[:,:,2:8]
    #loss_coordinates = mse_loss(coordinates,y_coordinates)
    #Get the -log likelihood
    #losses = Variable(torch.zeros(y.shape[0:2])) #.to(cuda0)
    losses_method = torch.zeros(y.shape[0:2],device = cuda0)
    losses_absolute = torch.zeros(y.shape[0:2],device = cuda0)
    losses = torch.zeros(y.shape[0:2],device = cuda0)
    
    for i in range(k):  # To Do: put k versions for method and absolute, add loss proportionally likelihood
        losses_method += coordinates_mix[:,:,i] * ce_loss(method[:,i*methods_n:(i+1)*methods_n,:],y_method)
        losses_absolute += coordinates_mix[:,:,i] * bce_loss(absolute[:,:,i],y_absolute)
        likelihood_z_x = gaussian_pdf(y[:,:,2:], coordinates_mu[:,:,i*t:(i+1)*t], coordinates_sigmasq[:,:, i*36:(i+1)*36])
        #print(likelihood_z_x)
        losses += coordinates_mix[:,:,i] * likelihood_z_x
    #print(torch.sum(losses_method),torch.sum(losses_absolute),torch.mean(-(torch.log(losses))))
    return (torch.mean(losses_method) + torch.mean(losses_absolute) + torch.mean(-(torch.log(losses))),losses)

In [ ]:
k = 30
methods_n =5 
model = LSTMArtist(12,400,methods_n,k,dropout=0.8).to(cuda0) #.to(cuda0) ,dropout=0.9
##################
ce_loss = nn.CrossEntropyLoss() # weight=weights, reduction = 'sum', idea - calciulate the weights based on frequency - less frequent get more weight
bce_loss = nn.BCELoss()
#mse_loss = nn.MSELoss()
lr = 0.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
#################
for epoch in range(20000):
    if epoch % 1000 == 0:
        print("Running epoch",epoch)
    total_loss = 0
    for i,batch in enumerate(x_batches):
        optimizer.zero_grad()
        method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix = model(batch)
        #print(coordinates_sigmasq)
        loss_sum,losses = calculate_loss(method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix,6,k,y_batches[i],ce_loss,bce_loss,methods_n)
        #print(">>>>>>>>>>>>>>>>>>>",losses)
        loss_sum.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1) #what param to set here? (grads are exploding without cliping)
        optimizer.step()
        #print(output.size())
        total_loss += loss_sum.item()
    if epoch % 1000 == 0:
        print(total_loss) #with 10 epochs 7720 2.36

In [ ]:
torch.save(model.state_dict(), 'model_MDM_v1.model')

In [ ]:
start_vector = x[:1,:1,:]

In [ ]:
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [ ]:
def random_sample(method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix,label_enc,onehot_enc):
    with torch.no_grad():
        mix_dist = Categorical(coordinates_mix[0,0,:])
        #print(coordinates_mix[0,0,:])
        mix_index = mix_dist.sample()
        #print(absolute.shape)
        method = method[:,:,mix_index*5:(mix_index+1)*5].cpu()
        absolute = absolute[mix_index].cpu()
        coordinates_mu = coordinates_mu.cpu()
        coordinates_sigmasq = coordinates_sigmasq.cpu()
        coordinates_mix = coordinates_mix.cpu()
        _,key = torch.max(method,2)
        str_key = [label_enc.inverse_transform(key)]
        #key = np.expand_dims(ey[0],0) #just to awoid a pesky warning, othewise don't needed
        onehot_key = onehot_enc.transform(key).toarray()
        absolute = [[torch.round(absolute)]]
        #get the mix component
        #mix_index = np.random.choice(list(range(k)),1, p=coordinates_mix[0,0,:])
        #print(coordinates_mix[0,0,:])
        
        mu = coordinates_mu[0,0,mix_index*6:(mix_index+1)*6]
        #print(mu)
        sigma = torch.sqrt(coordinates_sigmasq[0,0,mix_index])   
        #dist = Normal(mu, sigma) #sigma torch.tensor([0.000001])
        #cov = torch.eye(6)*coordinates_sigmasq[0,0,mix_index*6:(mix_index+1)*6]
        #print(coordinates_sigmasq.shape)
        cov = coordinates_sigmasq[0,0,mix_index*36:(mix_index+1)*36].reshape((6,6))
        #make the matrix positive-definite
        cov = torch.matmul(cov,cov.T)
        cov = cov + torch.eye(6)

        #print(cov)
        dist = MultivariateNormal(mu,cov.float())
        coordinates = dist.sample()
        #print(str_key,absolute,coordinates)
        return (np.concatenate((str_key,absolute,coordinates.reshape((1,6))),axis=1),np.concatenate((onehot_key,absolute,coordinates.reshape((1,6))),axis=1))

In [ ]:
start_vector

In [ ]:
# Generate the picture from the trained model
def generate_vectors(model,start_vector,iterations,label_enc,onehot_enc):
    vectors = [['m',0,0,0,0,0,0,0]]
    input_vecs = start_vector
    method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix = model(input_vecs)
    #print(coordinates_mu)
    #print("-------------")
    #print(coordinates_mix)
    vector,next_input = random_sample(method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix,label_enc,onehot_enc)
    vectors.append(vector)
    input_vecs = torch.cat((input_vecs,torch.tensor(np.expand_dims(next_input,0).astype('float'),dtype=torch.float,device=cuda0)),1)
    #print(">>>",input_vecs)
    for i in range(iterations):
        method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix = model(input_vecs)
        #print(">>>>>>>>>>",method,absolute,coordinates_mu,coordinates_sigmasq,coordinates_mix)
        vector,next_input = random_sample(method[:,-1:,:],absolute[-1],coordinates_mu[:,-1:,:],coordinates_sigmasq[:,-1:,:],coordinates_mix[:,-1:,:],label_enc,onehot_enc)
        vectors.append(vector)
        input_vecs = torch.cat((input_vecs,torch.tensor(np.expand_dims(next_input,0).astype('float'),dtype=torch.float,device=cuda0)),1)
        #print(input_vecs)
    return np.vstack(vectors)

In [ ]:
vectors = generate_vectors(model,start_vector,57,label_enc,enc)
vectors = vectors.astype('object')
vectors[:,1] = vectors[:,1].astype('float')
vectors[:,2] = vectors[:,2].astype('float')
vectors[:,3] = vectors[:,3].astype('float')
vectors[:,4] = vectors[:,4].astype('float')
vectors[:,5] = vectors[:,5].astype('float')
vectors[:,6] = vectors[:,6].astype('float')
vectors[:,7] = vectors[:,7].astype('float')
#vectors

In [ ]:
path_7 = generate_path(vectors)
dwg_7 = svgwrite.Drawing('test_180.svg') #,size = ("247", "148")
#dwg_4.viewbox(0,0,1000,100)
dwg_7.add(dwg_7.path( d=path_7,
    transform="scale(1), matrix(1.3333333,0,0,-1.3333333,100,100)"))
#))
dwg_7.save()
SVG('test_180.svg')